In [11]:
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input,Dense,TimeDistributed
from keras.layers import LSTM

In [12]:
batch_size = 32
num_classes = 2
epochs =5

In [13]:
import glob

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install -q keras
import keras

from keras.layers import Input,Lambda,Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [54]:
import glob
covid_path = glob.glob('/content/drive/MyDrive/COVID/*.png')
data = []
labels = []
i = 0
for file in covid_path:
  i = i+1
  if i > 500:
    break
  img = cv2.imread(file)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224, 224))
  data.append(img)
  labels.append('covid')
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [55]:
import glob
normal_path = glob.glob('/content/drive/MyDrive/NORMAL/*.png')
i=0
for file in normal_path:
  i= i + 1
  if i > 500:
    break
  img = cv2.imread(file)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (224, 224))
  data.append(img)
  labels.append('normal')
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [56]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [57]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [58]:
#embedding dimensions :
row_hidden = 128
col_hidden = 128

In [59]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(x_train, x_test, y_train, y_test) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [60]:
x_train = np.array(x_train)
x_train = x_train.reshape(x_train.shape[0],224,224,3)
x_train.shape

(800, 224, 224, 3)

In [61]:
x_test = np.array(x_test)
x_test = x_test.reshape(x_test.shape[0],224,224,3)
x_test.shape

(200, 224, 224, 3)

In [62]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [63]:
x_train = x_train/255
x_test = x_test/255

In [64]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [65]:
labels.shape

(1000, 2)

In [66]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=15,
	fill_mode="nearest")

In [67]:
row , col,pixel = x_train.shape[1:]

In [68]:
X= Input(shape=(row,col,pixel))

In [69]:
X

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [70]:
#encode a row of pixels using Timedisrtibuted wrapper
#TimeDistributed this wrapper applies a layer to every temporal slice of an input
#considered to be the temporal dimesions.Batch size = 32 
import tensorflow as tf
encoded_rows = TimeDistributed(LSTM(row_hidden))(X)
encoded_columns = LSTM(col_hidden)(encoded_rows) 

In [71]:
#Final predictions and model.
prediction = Dense(num_classes,activation='softmax')(encoded_columns)
model = Model(X,prediction)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy']) 

In [74]:
model.fit(x_train,y_train,batch_size=batch_size,epochs= 10 ,verbose =1 ,validation_data=(x_test,y_test))

Epoch 1/10
25/25 [==============================] - 27s 1s/step - loss: 0.2928 - accuracy: 0.8850 - val_loss: 0.2487 - val_accuracy: 0.9200
Epoch 2/10
25/25 [==============================] - 27s 1s/step - loss: 0.4681 - accuracy: 0.8000 - val_loss: 0.3952 - val_accuracy: 0.8450
Epoch 3/10
25/25 [==============================] - 26s 1s/step - loss: 0.4132 - accuracy: 0.8125 - val_loss: 0.3569 - val_accuracy: 0.8400
Epoch 4/10
25/25 [==============================] - 27s 1s/step - loss: 0.3671 - accuracy: 0.8487 - val_loss: 0.3142 - val_accuracy: 0.8800
Epoch 5/10
25/25 [==============================] - 27s 1s/step - loss: 0.3508 - accuracy: 0.8550 - val_loss: 0.3146 - val_accuracy: 0.9000
Epoch 6/10
25/25 [==============================] - 26s 1s/step - loss: 0.2942 - accuracy: 0.8863 - val_loss: 0.3344 - val_accuracy: 0.9000
Epoch 7/10
25/25 [==============================] - 26s 1s/step - loss: 0.2857 - accuracy: 0.8888 - val_loss: 0.2202 - val_accuracy: 0.9350
Epoch 8/10
25/25 [==

In [75]:
#Evaluation 
scores = model.evaluate(x_test,y_test)
print('Test Loss: ',scores[0])
print('Test Accuracy: ',scores[1])

7/7 [==============================] - 2s 224ms/step - loss: 0.2299 - accuracy: 0.9300
Test Loss:  0.22987380623817444
Test Accuracy:  0.9300000071525574
